In [1]:
from py2neo import authenticate, Graph, Relationship
g = Graph("http://localhost:7474/db/data/")
g.delete_all()
from py2neo import Node

In [2]:
import py2neo

NO_OF_DAYS, NO_OF_FLIGHTS, AP_MULTIPLE = 5, 1600, 1

airport_names = ["Dublin", "Paris","Toulouse", "Bordeaux", "London", "Malaga"]
airport_countries = ["Ireland","France","France","France","Britain","Spain"]

if AP_MULTIPLE > 1:
    airport_names = airport_names + [x + "_x" for x in airport_names]
    airport_countries = airport_countries + airport_countries
    print airport_names
    print airport_countries

In [3]:
airport_dict = {}

"""
    1. Create each Airport Node
    2. Add same country relationship to other Airports in the Same_Country
"""
for name, country in zip(airport_names, airport_countries):
    l = airport_dict.get(country, [])
    
    # 1. Create each Airport Node
    node = Node("Airport", name=name)
    g.create(node)
    
    # 2. Add same country relationship to other Airports in the Same_Country
    for other_airports_in_country in l:
        g.create(Relationship(node, "SAME_COUNTRY", other_airports_in_country))
    
    l.append(node)
    airport_dict[country] = l
    

WEEKDAYS = ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"]

print "CREATE", ",".join ( [ '({name}:Airport {{name:"{name}"}})'.format(name=a) for a in airport_names])

CREATE (Dublin:Airport {name:"Dublin"}),(Paris:Airport {name:"Paris"}),(Toulouse:Airport {name:"Toulouse"}),(Bordeaux:Airport {name:"Bordeaux"}),(London:Airport {name:"London"}),(Malaga:Airport {name:"Malaga"})


In [4]:

try:
    g.schema.create_uniqueness_constraint("Airport", "name")
except:
    pass

try:
    g.schema.create_uniqueness_constraint("Weekday", "weekday")
except:
    pass

try:
    g.schema.create_uniqueness_constraint("AirportDay", "date")
except:
    pass

In [5]:
import itertools

days = [ Node("Date", date = i) for i in range (1,NO_OF_DAYS+1)]

"""
    Also need CONNECTING, SHORT, MEDIUM, LONG, WEEKEND
"""
processed_days = []
for day in days:
    g.create( day )
    
    for pd in processed_days:
            """print pd
            print day
            print"""
            relationship = Relationship (pd, "Later", day)
            g.create( relationship )
            
    processed_days.append(day)

In [6]:
airports = list(g.find("Airport"))
airport_days = {}

for airport in airports:
    airport_name = airport['name']
    
    # Add numbers to represent dates
    for day in days:
        ad_name = data = airport_name + str(day['date'])
        ad = Node ("AirportDay", ad_name)
        g.create( ad )
        g.create( Relationship( airport, "AP", ad) )
        g.create( Relationship( day, "DA", ad) )
        airport_days[ad_name] = ad
            
for day in WEEKDAYS:
    g.create ( Node ( "Weekday" , day) )

In [7]:
print airport_days

{u'Malaga4': (f2f1e61:AirportDay:Malaga4), u'Malaga5': (fbf187b:AirportDay:Malaga5), u'Malaga2': (a03a22d:AirportDay:Malaga2), u'Malaga3': (c946413:AirportDay:Malaga3), u'Malaga1': (d541407:AirportDay:Malaga1), u'Bordeaux5': (b31409f:AirportDay:Bordeaux5), u'Bordeaux4': (afbfb2e:AirportDay:Bordeaux4), u'Bordeaux3': (f068e2c:AirportDay:Bordeaux3), u'Bordeaux2': (c2c4646:AirportDay:Bordeaux2), u'Bordeaux1': (c6cadcc:AirportDay:Bordeaux1), u'Paris1': (b270ac3:AirportDay:Paris1), u'Paris3': (d169cd3:AirportDay:Paris3), u'Paris2': (dabfe37:AirportDay:Paris2), u'Paris5': (fc658df:AirportDay:Paris5), u'Paris4': (cffceae:AirportDay:Paris4), u'London1': (d3eee4c:AirportDay:London1), u'London3': (ed98cad:AirportDay:London3), u'London2': (f1af88f:AirportDay:London2), u'London5': (fa64f68:AirportDay:London5), u'London4': (bc6111c:AirportDay:London4), u'Dublin1': (b96e8ca:AirportDay:Dublin1), u'Dublin3': (f1a1979:AirportDay:Dublin3), u'Dublin2': (fbae171:AirportDay:Dublin2), u'Dublin5': (cea05c9:Ai

In [8]:
weekdays = list(g.find("Weekday"))

for ai in airports:
    print ai

(dublin:Airport {name:"Dublin"})
(paris:Airport {name:"Paris"})
(toulouse:Airport {name:"Toulouse"})
(bordeaux:Airport {name:"Bordeaux"})
(london:Airport {name:"London"})
(malaga:Airport {name:"Malaga"})


In [9]:
airports[0].relationships()

()

In [10]:
import random
potential_flights = list ( itertools.permutations ( airport_names , 2) )
random.seed(117)

 

def create_flights():
    
    items = []
    
    tx = g.begin()

    for i in range(NO_OF_FLIGHTS):

        cost = random.randint(20,100)

        a = random.randint(1, NO_OF_DAYS)

        departure, arrival = random.choice(potential_flights)
        #to = g.find_one("AirportDay", 'data',departure+str(a))
        #dest = g.find_one("AirportDay", 'data', arrival+str(a))
        to = airport_days[departure+str(a)]
        dest = airport_days[arrival+str(a)]
        
        
        if i % 160 == 49:
            print a,to,dest
        weekday = random.choice(weekdays)
        flight_name = departure[:2]+'-'+arrival[:2]+'-'+str(a)
        flight_node = Node("Flight", flight_name = flight_name, flight_cost = cost)
        out, landing = Relationship(to, "Flight_Out", flight_node), Relationship(flight_node, "Flight_In", dest)
        
        items.extend([flight_node, out, landing])

        tx.create(flight_node )
        tx.create(out)
        tx.create(landing)
        tx.process()
        
    tx.commit()
        
    return items
    
def commit_flights(items):
        
    tx = g.begin()
    for f in items:
        try:
            tx.create(f)
        except e:
            print e
            tx.rollback()
            break
    else:    
        tx.commit()
  

In [11]:
# Process after creating 3 items = 42 seconds
# 43 seconds after 49th
# 49 seconds at end

In [12]:
%prun create_flights()

3 (c946413:AirportDay:Malaga3) (d169cd3:AirportDay:Paris3)
1 (b96e8ca:AirportDay:Dublin1) (fb3d76c:AirportDay:Toulouse1)
4 (a051b9c:AirportDay:Toulouse4) (bc6111c:AirportDay:London4)
4 (afbfb2e:AirportDay:Bordeaux4) (cffceae:AirportDay:Paris4)
5 (b31409f:AirportDay:Bordeaux5) (fbf187b:AirportDay:Malaga5)
3 (c946413:AirportDay:Malaga3) (cf7bc38:AirportDay:Toulouse3)
3 (cf7bc38:AirportDay:Toulouse3) (f068e2c:AirportDay:Bordeaux3)
2 (c2c4646:AirportDay:Bordeaux2) (fbae171:AirportDay:Dublin2)
4 (a051b9c:AirportDay:Toulouse4) (f2f1e61:AirportDay:Malaga4)
3 (c946413:AirportDay:Malaga3) (f1a1979:AirportDay:Dublin3)
 

In [16]:
%%time
# Airport connected by AP to AirportDay
flights_from_malaga = """
MATCH (malaga:Airport) - [:AP] -> (ad1:AirportDay) - [out1:Flight_Out] -> (f:Flight) - [in1:Flight_In] -> (ad2:AirportDay) <- 
[:AP] - (dest:Airport)
,
(dest:Airport) - [:AP] -> (ad3:AirportDay) - [out2:Flight_Out] -> (f2:Flight) - 
[in2:Flight_In] -> (ad4:AirportDay) <- [:AP] - (malaga)
,
(ad2)<-[da:DA]-(d1:Date)-[:Later]->(d2:Date)-[:DA] -> (ad3)
WHERE malaga.name='Malaga' 
RETURN  *
"""

results = list(g.run(flights_from_malaga))

print "NO OF FLIGHTS", NO_OF_FLIGHTS
print "NO OF AIRPORTS"
print "NO OF RESULTS", len(results)

NO OF FLIGHTS 1600
NO OF AIRPORTS
NO OF RESULTS 390
Wall time: 3min 22s


In [21]:
%%time
# Airport connected by AP to AirportDay
flights_from_malaga = """
MATCH (malaga:Airport) - [:AP] -> (ad1:AirportDay) - [out1:Flight_Out] -> (f:Flight) - [in1:Flight_In] -> (ad2:AirportDay) <- 
[:AP] - (dest:Airport)
,
(ad2)<-[da:DA]-(d1:Date)-[:Later]->(d2:Date)-[da2:DA] -> (ad3:AirportDay) <- [:AP] - (dest:Airport)
,
(ad3:AirportDay) - [out2:Flight_Out] -> (f2:Flight) - [in2:Flight_In] -> (ad4:AirportDay) <- [:AP] - (malaga)

WHERE malaga.name='Malaga' 
RETURN  *
"""

results = list(g.run(flights_from_malaga))

print "NO OF FLIGHTS", NO_OF_FLIGHTS
print "NO OF AIRPORTS"
print "NO OF RESULTS", len(results)

NO OF FLIGHTS 1600
NO OF AIRPORTS
NO OF RESULTS 4887
Wall time: 3min 39s


In [ ]:
#
from itertools import groupby



print "There are {0} results \n" .format ( str(len(results)) ) 

for r in results:
    for k in r:
        print k
        
    print "\n================\n"

import operator

results_dict = {}

for key, group in groupby(results, lambda result: result['dest']['name']):
    key_dict = results_dict.get(key, [])
    key_dict.extend( group )
    results_dict[key] = key_dict
    
for key in results_dict.keys():
    print key, min ( results_dict[key] , key = operator.itemgetter(1) )['f']['flight_cost']

There are 4887 results 

(f2f1e61:AirportDay:Malaga4)
(afbfb2e:AirportDay:Bordeaux4)
(b31409f:AirportDay:Bordeaux5)
(fbf187b:AirportDay:Malaga5)
(b249745:Date {date:4})
(c929012:Date {date:5})
(b249745)-[:DA]->(afbfb2e)
(bordeaux:Airport {name:"Bordeaux"})
(dc548e7:Flight {flight_cost:27,flight_name:"Ma-Bo-4"})
(b554a3d:Flight {flight_cost:95,flight_name:"Bo-Ma-5"})
(dc548e7)-[:Flight_In]->(afbfb2e)
(b554a3d)-[:Flight_In]->(fbf187b)
(malaga:Airport {name:"Malaga"})
(f2f1e61)-[:Flight_Out]->(dc548e7)
(b31409f)-[:Flight_Out]->(b554a3d)


(f2f1e61:AirportDay:Malaga4)
(afbfb2e:AirportDay:Bordeaux4)
(b31409f:AirportDay:Bordeaux5)
(fbf187b:AirportDay:Malaga5)
(b249745:Date {date:4})
(c929012:Date {date:5})
(b249745)-[:DA]->(afbfb2e)
(bordeaux:Airport {name:"Bordeaux"})
(dc548e7:Flight {flight_cost:27,flight_name:"Ma-Bo-4"})
(f84f94e:Flight {flight_cost:73,flight_name:"Bo-Ma-5"})
(dc548e7)-[:Flight_In]->(afbfb2e)
(f84f94e)-[:Flight_In]->(fbf187b)
(malaga:Airport {name:"Malaga"})
(f2f1e61)-[:F

# Not sure why identifiers != name
res = list(g.run("""
MATCH (d1:Date)-[:Later]-(d2:Date)
RETURN *
"""))

for re in res:
    print re